In [1]:
from tool import *
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side

In [2]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    spdb = pickle.load(f)

with open('D:/Data/contact_window.db', 'rb') as f:
    pic = pickle.load(f)

with open('D:/Data/ODM PO.db', 'rb') as f:
    po = pickle.load(f)
    
with open('D:/Data/TCL PP_new.bin', 'rb') as f:
    pp_db = pickle.load(f)

with open(f'D:/Data/ODM_SR.db', 'rb') as f:
    sr = pickle.load(f)

with open('D:/Data/sitemap.db', 'rb') as f:
    ste = pickle.load(f)

In [3]:
ver_table = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index()
ver_table = ver_table[ver_table['From Site'].str.contains('TCL')]
ver_table.iloc[-30:, ]

,Planweek,From Site,Category,Created_at,SP
469,2023-12-04(W49),OS_TCL_CN_P,Latest26w,202312041556,403883.0
470,2023-12-04(W49),OS_TCL_CN_P,Latest26w,202312050956,404187.0
471,2023-12-04(W49),OS_TCL_CN_P,Latest26w,202312070937,404149.0
502,2023-12-11(W50),OS_TCL_CN_P,Final26w,202312130843,389916.0
503,2023-12-11(W50),OS_TCL_CN_P,Final52w,202312121417,565695.0
504,2023-12-11(W50),OS_TCL_CN_P,Latest26w,202312110909,382170.0
505,2023-12-11(W50),OS_TCL_CN_P,Latest26w,202312111514,390428.0
506,2023-12-11(W50),OS_TCL_CN_P,Latest26w,202312121117,390172.0
507,2023-12-11(W50),OS_TCL_CN_P,Latest26w,202312141259,389916.0
538,2023-12-18(W51),OS_TCL_CN_P,Final26w,202312200913,421194.0


In [4]:
i = 628
sp = pd.merge(spdb, spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i], :4])
sp = sp[['To Site', 'Mapping Model.Suffix', 'Rep PMS', 'Week Name', 'Category', 'SP']]
sp = sp.rename(columns={'SP':'QTY'})
sp['Category'] = '2_SP'

In [6]:
sp['Week Name'].min()

'2023-12-04(W49)'

In [5]:
sp = sp[sp['Week Name'] >= '2023-10-16(W42)'] # 10/16W SP부터 계산하기 위해

In [6]:
# Open PO 데이터프레임 생성
opo = pd.merge(po.loc['OS_TCL_CN_P'].reset_index(), sr.loc['OS_TCL_CN_P'].groupby('PO No.').sum('Ship')['Ship'].reset_index(), how='left').fillna(0)
opo['Week Name'] = opo['RSD'].map(get_weekname)
opo['Ship'] = opo['Ship'].astype(int)
opo['QTY'] = opo['PO'] - opo['Ship'] # PO에서 선적된 수량 차감하여 잔량 PO 수량 계산
opo = opo[opo['QTY'] > 0]
opo['Category'] = '3_PO'
opo['Rep PMS'] = opo['Mapping Model.Suffix'].map(sp[['Mapping Model.Suffix', 'Rep PMS']].drop_duplicates().set_index('Mapping Model.Suffix')['Rep PMS']) # PMS 모델명 추가
opo = opo[['To Site', 'Mapping Model.Suffix', 'Rep PMS', 'Week Name', 'Category', 'QTY']]
ship_to_map = {'LG Electronics United Kingdom Ltd.':'EEUK', 'LG ELECTRONICS DEUTSCHLAND.GMBH.':'EEDG', 'ENUS_TX':'ENUS_V', 'ENUS_CA':'ENUS_V'}
opo['To Site'] = opo['To Site'].replace(ship_to_map)
ref_wk = spdb.groupby(['Planweek', 'Created_at', 'Category', 'From Site']).sum('SP').reset_index().iloc[i]['Planweek']
opo.loc[opo['Week Name'] < ref_wk, 'Week Name'] = ref_wk

In [7]:
# 최신 Version의 PP와 직전 PP와의 생산수량 변경 내역 계산
pp1 = pp_db[pp_db['Updated_at'] == pp_db['Updated_at'].unique()[-1]].reset_index(drop=True) # 최신 PP 를 가져옴
pp2 = pp_db[pp_db['Updated_at'] == pp_db['Updated_at'].unique()[-2]].reset_index(drop=True)# 이전 Version의 PP를 가져옴
pp_sum = pd.concat([pp1, pp2]).reset_index(drop=True)
pp_sum['Month'] = pp_sum['PP Date'].apply(lambda x:f'{x.isocalendar().year}-{get_month_from_date(x):02d}')
pp_sum['Week Name'] = pp_sum['PP Date'].map(get_weekname)
pp_sum['Series'] = pp_sum['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0])
pp_sum['Day'] = pp_sum['PP Date'].dt.strftime('%a')
pp_sum['Date'] = pp_sum['PP Date'].dt.strftime('%m/%d')
pp_sum = pp_sum.merge(ste[['To Site', 'Region', 'Country']], how='left')

gap_month = pp_sum.groupby(['Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'TCL BOM', 'Month', 'Week Name', 'Date', 'Day', 'Updated_at']).sum('QTY').unstack('Updated_at').fillna(0).astype(int)
gap_month['Gap'] = gap_month[gap_month.columns[-1]] - gap_month[gap_month.columns[0]]
gap_month = gap_month.unstack(['Month', 'Week Name']).dropna(how='all', axis=1).fillna(0).astype(int)
gap_month['Sum'] = gap_month['Gap'].sum(axis=1)
# gap_month.loc[('', '', '', '', '', 'Sum'), :] = gap_month.sum()
gap_month = gap_month.astype(int)

IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
pp_db

In [64]:
# TCL 신규 PP 양식의 경우
pp_db['Week Name'] = pp_db['PP Date'].map(get_weekname)
pp_db = pp_db[pp_db['Week Name'] >= sp['Week Name'].min()]
pp_db['Category'] = '4_PP'
pp = pp_db[['To Site', 'Mapping Model.Suffix', 'Rep PMS', 'Week Name', 'Category', 'QTY']].copy()

KeyError: "['To Site', 'Mapping Model.Suffix'] not in index"

In [67]:
pp1['Week Name'] = pp1['PP Date'].map(get_weekname)
pp1 = pp1[pp1['Week Name'] >= sp['Week Name'].min()]
pp1['Category'] = '4_PP'
pp = pp1[['To Site', 'Mapping Model.Suffix', 'Week Name', 'Category', 'QTY']].copy()

In [68]:
p1 = pd.concat([sp, opo, pp])[['To Site', 'Mapping Model.Suffix']].drop_duplicates().reset_index(drop=True)
p1['Category'] = '1_BOH'
psi = pd.concat([p1, p1.drop('Category', axis=1)]).fillna('2_SP').reset_index(drop=True)
psi = pd.concat([psi, p1.drop('Category', axis=1)]).fillna('3_PO').reset_index(drop=True)
psi = pd.concat([psi, p1.drop('Category', axis=1)]).fillna('4_PP').reset_index(drop=True)
psi = pd.concat([psi, p1.drop('Category', axis=1)]).fillna('5_ETD').reset_index(drop=True)
psi = pd.concat([psi, p1.drop('Category', axis=1)]).fillna('6_SP Gap').reset_index(drop=True)

psi = psi.set_index(['To Site', 'Mapping Model.Suffix', 'Category'])
min_wk = pd.concat([sp, opo, pp])['Week Name'].min()
max_wk = pd.concat([sp, opo, pp])['Week Name'].max()
wk = min_wk
wklist = [get_weekname_from(wk, i) for i in range(60)]
wklist = wklist[:wklist.index(max_wk)+1]
psi[wklist] = 0
psi.update(sp.pivot_table('QTY', index=['To Site', 'Mapping Model.Suffix', 'Category'], columns='Week Name', aggfunc=sum).fillna(0).astype(int))
psi.update(opo.pivot_table('QTY', index=['To Site', 'Mapping Model.Suffix', 'Category'], columns='Week Name', aggfunc=sum).fillna(0).astype(int))
psi.update(pp.pivot_table('QTY', index=['To Site', 'Mapping Model.Suffix', 'Category'], columns='Week Name', aggfunc=sum).fillna(0).astype(int))
psi = psi.convert_dtypes().sort_index()

In [69]:
boh = pd.read_excel(get_filename(), skiprows=1)
boh = boh[boh['Category'] == '1_BOH']
boh = boh[['To Site', 'Mapping Model.Suffix', 'Category', min_wk]]
boh = boh[boh[min_wk] > 0].set_index(['To Site', 'Mapping Model.Suffix', 'Category'])
boh

,,,2023-09-04(W36)
To Site,Mapping Model.Suffix,Category,
EMTK,22MR410-B.APDQ,1_BOH,840


In [70]:
psi.update(boh)

In [71]:
for site, model in p1[['To Site', 'Mapping Model.Suffix']].values:
    for week in psi.columns:
        if week == psi.columns[0]: # 계산하는 Week가 시작하는 Week 일 경우
            current_boh = psi.loc[(site, model, '1_BOH'), week] # 금주 boh
            current_sp = psi.loc[(site, model, '2_SP'), week] # 금주 sp
            if current_boh > current_sp :
                psi.loc[(site, model, '5_ETD'), week] = current_sp
            else:
                psi.loc[(site, model, '5_ETD'), week] = current_boh
            psi.loc[(site, model, '6_SP Gap'), week] = psi.loc[(site, model, '5_ETD'), week] - current_sp
        else:
            last_boh = psi.loc[(site, model, '1_BOH'), get_weekname_from(week, -1)]
            last_pp = psi.loc[(site, model, '4_PP'), get_weekname_from(week, -1)]
            last_etd = psi.loc[(site, model, '5_ETD'), get_weekname_from(week, -1)]
            psi.loc[(site, model, '1_BOH'), week] =  last_boh + last_pp - last_etd
            current_boh = psi.loc[(site, model, '1_BOH'), week]
            current_sp = psi.loc[(site, model, '2_SP'), week]
            last_gap = psi.loc[(site, model, '6_SP Gap'), get_weekname_from(week, -1)]
            if current_boh  > current_sp - last_gap:
                psi.loc[(site, model, '5_ETD'), week] = current_sp - last_gap
            else:
                psi.loc[(site, model, '5_ETD'), week] = current_boh
            psi.loc[(site, model, '6_SP Gap'), week] = psi.loc[(site, model, '5_ETD'), week] - current_sp + last_gap
psi = pd.merge(psi.reset_index(), ste[['To Site', 'Region', 'Country']], how='left').set_index(['Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'Category'])
psi = psi.reset_index()
psi.insert(4, 'TCL BOM', psi['Mapping Model.Suffix'].map(pp_db[['Mapping Model.Suffix', 'TCL BOM']].groupby('Mapping Model.Suffix')['TCL BOM'].max()))
psi = psi.set_index(['Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'TCL BOM', 'Category'])

In [72]:
# pp_daily = pp_sum.pivot_table('QTY', index=['Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'Date', 'Day'], columns=['Updated_at', 'Month'], aggfunc=sum).fillna('-')
update_time = datetime.datetime.now().strftime('%Y%m%d%H')
filename = f'{update_time}_TCL Monitor PSI.xlsx'
with pd.ExcelWriter(f'D:/MNT/TCL/PSI/{filename}') as writer:
    psi.to_excel(writer, sheet_name='TCL PSI', index=True, merge_cells=False, startrow=1)
    gap_month.to_excel(writer, sheet_name='PP Summary', startrow=1)

In [73]:
wb = load_workbook(f'D:/MNT/TCL/PSI/{filename}')

color = PatternFill(fgColor='9999FF', fill_type='solid') # Blue
big_size = Font(name='맑은 고딕', size=20, bold=True, color='000099')
middle_size = Font(name='맑은 고딕', size=15, bold=True, color='000099')

border_style = Side(border_style="thin", color="000000")
border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)

In [74]:
sh = wb[wb.sheetnames[0]]
sh['A1'] = f'■ TCL Monitor PSI_updated at : {update_time}'
sh['A1'].font = middle_size
sh['F1'] = 'Sub TTL'
for i in range(7, sh.max_column+1):
    col_letter = get_column_letter(i)
    sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}3:{col_letter}{sh.max_row})'
    sh.cell(row=1, column=i).border = border
    sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
for cell in sh[2]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=3, min_col=7, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.freeze_panes = 'G3'
sh.column_dimensions['B'].width = 10
sh.column_dimensions['D'].width = 17.5
sh.column_dimensions['E'].width = 23
sh.column_dimensions['F'].width = 10

sh = wb[wb.sheetnames[1]]
sh['A1'] = f'■ TCL PP Gap from previous version'
sh['A1'].font = middle_size
sh['G1'] = 'Sub TTL'
for i in range(8, sh.max_column+1):
    col_letter = get_column_letter(i)
    sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}7:{col_letter}{sh.max_row})'
    sh.cell(row=1, column=i).border = border
    sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
for row in sh.iter_rows(min_row=2, min_col=len(gap_month.index[0]) ,max_row=2+len(gap_month.columns[0])-1, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
merge_range = f'{get_column_letter(sh.max_column)}{2}:{get_column_letter(sh.max_column)}{2+len(gap_month.columns[0])-1}'
sh.merge_cells(merge_range)
for cell in sh[merge_range][0]:
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=6, min_col=1 ,max_row=6, max_col=len(gap_month.index[0])):
    for cell in row:
        cell.fill = color
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=7, min_col=8, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.freeze_panes = 'H7'
sh.column_dimensions['B'].width = 10
sh.column_dimensions['D'].width = 17.5
sh.column_dimensions['E'].width = 23
sh.column_dimensions['G'].width = 10.5

for name in wb.sheetnames:
    wb[name].sheet_view.showGridLines = False
wb.save(f'D:/MNT/TCL/PSI/{filename}')
wb.close()

In [382]:
psi.columns.name = 'Week Name'

In [386]:
psi = psi.stack('Week Name').reset_index()
psi = psi.rename(columns={0:'QTY'})

In [388]:
psi['Month'] = psi['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')

In [389]:
psi

,Region,Country,To Site,Mapping Model.Suffix,Category,Week Name,QTY,Month
0,SEA,Philippines,EACM,22MR410-B.APHQ,1_BOH,2023-08-07(W32),0,2023-08
1,SEA,Philippines,EACM,22MR410-B.APHQ,1_BOH,2023-08-14(W33),0,2023-08
2,SEA,Philippines,EACM,22MR410-B.APHQ,1_BOH,2023-08-21(W34),0,2023-08
3,SEA,Philippines,EACM,22MR410-B.APHQ,1_BOH,2023-08-28(W35),0,2023-08
4,SEA,Philippines,EACM,22MR410-B.APHQ,1_BOH,2023-09-04(W36),0,2023-09
...,...,...,...,...,...,...,...,...
13567,SCA,Chile,ESCL_STG,27MR400-B.AWHQ,6_SP Gap,2024-01-01(W01),-2048,2024-01
13568,SCA,Chile,ESCL_STG,27MR400-B.AWHQ,6_SP Gap,2024-01-08(W02),-2048,2024-01
13569,SCA,Chile,ESCL_STG,27MR400-B.AWHQ,6_SP Gap,2024-01-15(W03),-2048,2024-01
13570,SCA,Chile,ESCL_STG,27MR400-B.AWHQ,6_SP Gap,2024-01-22(W04),-2048,2024-01


In [205]:
sp = sp.pivot_table('QTY', index=['Region', 'Country', 'To Site', 'Mapping Model.Suffix'], columns='Week Name', aggfunc=sum).fillna(0).astype(int)

In [216]:
pd.concat([sp, pp]).pivot_table('QTY', index=['Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'Category'], columns='Week Name', aggfunc=sum)

Week Name                                               2023-08-07(W32)  \
Region Country  To Site  Mapping Model.Suffix Category                    
EU     Czech    EEPL_CZ  22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       France   EEFS     22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       Hungary  EEMK     22MR410-B.AEUQ       PP                  840.0   
...                                                                 ...   
SEA    Thailand EATH     22MR410-B.ATMQ       SP                    NaN   
                         27MR400-B.ATMQ       SP                    NaN   
       Vietnam  EAVH_HCM 22MR410-B.ATVQ       SP                    NaN   
                         27MR400-B.ATVQ       SP                    NaN   
                EAVH_HN  27MR400-B.ATVQ       SP                    NaN   

Week Name                                               2023-08-14(W33)  \
Region Country  To Site  Mapping Model.Suffix Category                    
EU     Czech    EEPL_CZ  22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       France   EEFS     22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       Hungary  EEMK     22MR410-B.AEUQ       PP                    NaN   
...                                                                 ...   
SEA    Thailand EATH     22MR410-B.ATMQ       SP                 2184.0   
                         27MR400-B.ATMQ       SP                    NaN   
       Vietnam  EAVH_HCM 22MR410-B.ATVQ       SP                    NaN   
                         27MR400-B.ATVQ       SP                    NaN   
                EAVH_HN  27MR400-B.ATVQ       SP                    NaN   

Week Name                                               2023-08-21(W34)  \
Region Country  To Site  Mapping Model.Suffix Category                    
EU     Czech    EEPL_CZ  22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       France   EEFS     22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       Hungary  EEMK     22MR410-B.AEUQ       PP                    NaN   
...                                                                 ...   
SEA    Thailand EATH     22MR410-B.ATMQ       SP                 2184.0   
                         27MR400-B.ATMQ       SP                    NaN   
       Vietnam  EAVH_HCM 22MR410-B.ATVQ       SP                    NaN   
                         27MR400-B.ATVQ       SP                    NaN   
                EAVH_HN  27MR400-B.ATVQ       SP                    NaN   

Week Name                                               2023-08-28(W35)  \
Region Country  To Site  Mapping Model.Suffix Category                    
EU     Czech    EEPL_CZ  22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       France   EEFS     22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       Hungary  EEMK     22MR410-B.AEUQ       PP                    NaN   
...                                                                 ...   
SEA    Thailand EATH     22MR410-B.ATMQ       SP                    NaN   
                         27MR400-B.ATMQ       SP                    NaN   
       Vietnam  EAVH_HCM 22MR410-B.ATVQ       SP                    NaN   
                         27MR400-B.ATVQ       SP                    NaN   
                EAVH_HN  27MR400-B.ATVQ       SP                    NaN   

Week Name                                               2023-09-04(W36)  \
Region Country  To Si

In [133]:
pp

,Mapping Model.Suffix,To Site,QTY,Week Name,Region,Country,Category
0,22MR410-B.AEUQ,EEES,840,2023-08-21(W34),EU,Spain,PP
1,22MR410-B.AEUQ,EEMK,840,2023-08-07(W32),EU,Hungary,PP
2,22MR410-B.APDQ,EMTK,840,2023-08-14(W33),MEA,Turkey,PP
3,22MR410-B.APDQ,EMTK,840,2023-08-28(W35),MEA,Turkey,PP
4,22MR410-B.ATIQ,EAIN_DC,2184,2023-08-28(W35),SEA,Indonesia,PP
5,22MR410-B.ATMQ,EATH,2184,2023-08-14(W33),SEA,Thailand,PP
6,22MR410-B.ATMQ,EATH,2184,2023-08-21(W34),SEA,Thailand,PP
7,22MR410-B.AWPQ,ESCB,840,2023-08-28(W35),SCA,Columbia,PP


In [134]:
sp

,To Site,Region,Country,Mapping Model.Suffix,Week Name,Category,QTY
0,EACM,SEA,Philippines,22MR410-B.APHQ,2023-10-09(W41),SP,840
1,EACM,SEA,Philippines,22MR410-B.APHQ,2023-11-27(W48),SP,840
2,EACM,SEA,Philippines,22MR410-B.APHQ,2024-01-01(W01),SP,840
3,EACM,SEA,Philippines,24MR400-B.APHQ,2023-10-02(W40),SP,3060
4,EACM,SEA,Philippines,24MR400-B.APHQ,2023-10-30(W44),SP,1530
...,...,...,...,...,...,...,...
263,ESCB,SCA,Columbia,24MR400-B.AWPQ,2023-12-04(W49),SP,800
264,ESCL_STG,SCA,Chile,22MR410-B.AWHQ,2023-09-18(W38),SP,1680
265,ESCL_STG,SCA,Chile,22MR410-B.AWHQ,2023-10-16(W42),SP,840
266,ESCL_STG,SCA,Chile,27MR400-B.AWHQ,2023-09-18(W38),SP,1024


In [136]:
psi = pd.concat([sp, pp])

In [137]:
psi.head(3)

,To Site,Region,Country,Mapping Model.Suffix,Week Name,Category,QTY
0,EACM,SEA,Philippines,22MR410-B.APHQ,2023-10-09(W41),SP,840
1,EACM,SEA,Philippines,22MR410-B.APHQ,2023-11-27(W48),SP,840
2,EACM,SEA,Philippines,22MR410-B.APHQ,2024-01-01(W01),SP,840


In [140]:
psi.pivot_table('QTY', index=['Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'Category'], columns='Week Name', aggfunc=sum).fillna(0).astype(int).to_clipboard()

In [98]:
pd.concat([sp[['To Site', 'Region', 'Country', 'Mapping Model.Suffix', 'Week Name', 'SP']], pp])

,To Site,Region,Country,Mapping Model.Suffix,Week Name,SP,PP
0,EACM,SEA,Philippines,22MR410-B.APHQ,2023-10-09(W41),840.0,NaN
1,EACM,SEA,Philippines,22MR410-B.APHQ,2023-11-27(W48),840.0,NaN
2,EACM,SEA,Philippines,22MR410-B.APHQ,2024-01-01(W01),840.0,NaN
3,EACM,SEA,Philippines,24MR400-B.APHQ,2023-10-02(W40),3060.0,NaN
4,EACM,SEA,Philippines,24MR400-B.APHQ,2023-10-30(W44),1530.0,NaN
...,...,...,...,...,...,...,...
3,EMTK,MEA,Turkey,22MR410-B.APDQ,2023-08-28(W35),NaN,840.0
4,EAIN_DC,SEA,Indonesia,22MR410-B.ATIQ,2023-08-28(W35),NaN,2184.0
5,EATH,SEA,Thailand,22MR410-B.ATMQ,2023-08-14(W33),NaN,2184.0
6,EATH,SEA,Thailand,22MR410-B.ATMQ,2023-08-21(W34),NaN,2184.0


In [21]:
pic[pic['Subsidiary'].str.contains('SJ')]

,Team,SCM,Position,Subsidiary,Email
40,SCM,오석현,책임,LGESJ,seokhyun.oh@lge.com
89,MKT,변영탁,사원,LGESJ,youngtak.byun@lge.com


In [5]:
pic[pic['Subsidiary'].str.contains('DG')]

,Team,SCM,Position,Subsidiary,Email
7,SCM,송명규,책임,LGEDG,danny.song@lge.com
61,MKT,윤석규,책임,LGEDG,sk.yoon@lge.com


In [92]:
sp = sp[sp['From Site'].str.contains('TCL')]
sp = sp[sp['Frozen'] == 'Y']
sp = pd.merge(sp, site[['To Site', 'Subsidiary', 'Country']], how='left').drop(columns=['Planweek', 'Created_at', 'Division', 'Rep PMS', 'Category', 'Frozen'])

In [93]:
pic['Subsidiary'] = pic['Subsidiary'].apply(lambda x:x.split(' ')[0])

In [94]:
po = po.reset_index()
po['Week Name'] = po['RSD'].apply(get_weekname)
po = po[po['Cancel'] == 'N']

In [95]:
sp = pd.merge(sp, po.drop(columns=['Cancel', 'Issued Date', 'RSD', 'Method', 'Price Term', 'Currency', 'Payment Term']), how='left')

In [97]:
sp.to_clipboard()

In [58]:
pic[pic['Team'] == 'MKT']

,Team,SCM,Position,Subsidiary,Email
55,MKT,장진현,선임,LGECI,jinhyeon.jang@lge.com
56,MKT,민병우,책임,LGEUS,joseph.min@lge.com
57,MKT,박준성,책임,LGEUS,joonsung23.park@lge.com
58,MKT,김재영,사원,LGEJP,jaeyoung3.kim@lge.com
59,MKT,오세림,사원,LGEBN,selim1.oh@lge.com
60,MKT,김한주,책임,LGECZ,hanju.kim@lge.com
61,MKT,윤석규,책임,LGEDG,sk.yoon@lge.com
62,MKT,김원중,책임,LGEES,wj.kim@lge.com
63,MKT,김지환,책임,LGEFS,jacky5.kim@lge.com
64,MKT,이택준,선임,LGEHS,taekjun.lee@lge.com
